### Import of libraries

In [4]:
import pandas as pd
import numpy as np

In [ ]:
from os import listdir
from os.path import isfile, join

In [409]:
from tqdm import tqdm

In [ ]:
from html2image import Html2Image
hti = Html2Image()

In [473]:
!pip install Pillow -q

You should consider upgrading via the '/Users/lisa/projects/MTReviews/venv/bin/python -m pip install --upgrade pip' command.


In [474]:
from PIL import Image

### Functions

In [482]:
# Image cropping
def bbox(im):
    a = np.array(im)[:,:,:3]  # keep RGB only
    m = np.any(a != [255, 255, 255], axis=2)
    coords = np.argwhere(m)
    y0, x0, y1, x1 = *np.min(coords, axis=0), *np.max(coords, axis=0)
    return (x0, y0, x1+1, 500)

In [426]:
# Visualisation in Jupyter
def visualising(text):
    split = sentence_splitting(clearing_junk(text))
    pred = aspect_pipeline(split, top_k=3)
    
    res = []

    for s in range(len(split)):
        res.append([split[s],pred[s][0]['label'],str(round(pred[s][0]['score']*100))+'%'])
        
    words = [r[0].strip(' ') for r in res]

    words_scores = []
    for w in range(len(words)):
        words_scores.append(words[w])
        words_scores.append(res[w][2].upper())

    text = ' '.join(words_scores)
    ents = []

    strt = 0
    for r in res:
        endd = strt + len(r[0].strip(' ')) + 1
        ents.append({"start": strt, "end": endd, 'label': r[1].upper()})
        strt += len(r[0].strip(' ')) + 1 + 4


    man = {
        "text": text,
        "ents": ents,
        "title": None
    }
    
    #svg = displacy.render(man, manual=True, style="ent", jupyter=True, options={'colors':aspect_mapper2, 'ents': aspects})
    svg = displacy.render(man, manual=True, style="ent", jupyter=True, options={'colors':aspect_mapper2, 'ents': aspects})

    
    return svg

In [460]:
# Visualisation for PNG saving
def visualising2(text):
    split = sentence_splitting(clearing_junk(text))
    pred = aspect_pipeline(split, top_k=3)
    
    res = []

    for s in range(len(split)):
        res.append([split[s],pred[s][0]['label'],str(round(pred[s][0]['score']*100))+'%'])
        
    words = [r[0].strip(' ') for r in res]

    words_scores = []
    for w in range(len(words)):
        words_scores.append(words[w])
        words_scores.append(res[w][2].upper())

    text = ' '.join(words_scores)
    ents = []

    strt = 0
    for r in res:
        endd = strt + len(r[0].strip(' ')) + 1
        ents.append({"start": strt, "end": endd, 'label': r[1].upper()})
        strt += len(r[0].strip(' ')) + 2 + len(r[2].upper())


    man = {
        "text": text,
        "ents": ents,
        "title": None
    }
    
    
    #svg = displacy.render(man, manual=True, style="ent", jupyter=True, options={'colors':aspect_mapper2, 'ents': aspects})
    svg = displacy.render(man, manual=True, style="ent", jupyter=False, options={'colors':aspect_mapper2, 'ents': aspects},
                         page=True, minify=True)

    
    return svg

### Visualising

In [414]:
# On longer non-positive reviews for demo
ids = revs[(revs.DETAIL_RATING<5)&(revs.DETAIL_CONTENT.str.len()>100)].index.to_list()

In [415]:
len(ids)

56701

In [408]:
i = np.random.choice(ids)
print(i)
visualising(str(revs.loc[i, 'content']))

255666


In [416]:
sample = revs[(revs.DETAIL_RATING<5)&(revs.DETAIL_CONTENT.str.len()>100)].groupby(
    'DETAIL_RATING').sample(125, random_state=42)

In [417]:
len(sample)

500

In [418]:
ids = sample['index'].to_list()

### Saving visualisations as PNG

In [486]:
mypath = './review_vis/demo_review_vis_mid/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [488]:
for f in tqdm([f for f in onlyfiles if f.endswith('html')]):
    # imgkit.from_file(mypath+f, f.replace("html", "jpg"))
    hti.screenshot(
    html_file=mypath+f, save_as=f.replace("html", "png"))
    im = Image.open((f).replace("html", "png"))
    print(bbox(im))
    im2 = im.crop(bbox(im))
    print(bbox(im2))
    im2.save((mypath+f).replace("html", "png"))


  0%|          | 0/5 [00:00<?, ?it/s]103286 bytes written to file /Users/lisa/projects/MTReviews/vis_MRDB10754022275185_rate3_plot.png


(0, 0, 3840, 500)
(0, 0, 3840, 500)


 20%|██        | 1/5 [00:03<00:15,  3.82s/it]172286 bytes written to file /Users/lisa/projects/MTReviews/vis_APJO89470119407139_rate2_plot.png


(0, 0, 3840, 500)
(0, 0, 3840, 500)


 40%|████      | 2/5 [00:07<00:12,  4.02s/it]212066 bytes written to file /Users/lisa/projects/MTReviews/vis_ADUP98489228440688_rate4_plot.png


...

 60%|██████    | 3/5 [00:11<00:07,  4.00s/it][0601/235009.474697:ERROR:command_buffer_proxy_impl.cc(131)] ContextResult::kTransientFailure: Failed to send GpuControl.CreateCommandBuffer.
106158 bytes written to file /Users/lisa/projects/MTReviews/vis_AJUM52128422481383_rate1_plot.png


(0, 0, 3840, 500)
(0, 0, 3840, 500)


 80%|████████  | 4/5 [00:19<00:05,  5.49s/it]215786 bytes written to file /Users/lisa/projects/MTReviews/vis_AQUW34117257982599_rate4_plot.png


(0, 0, 3840, 500)
(0, 0, 3840, 500)


100%|██████████| 5/5 [00:26<00:00,  5.31s/it]
